# Monocular Visual Odometry

Akshay Aggarwal

This is a simple implementation of monocular visual odometry using optical flow. the dataset used here is provided by [KITTI](http://www.cvlibs.net/datasets/kitti/eval_odometry.php)
For simplicity ( and efficieny), I have used the grayscale dataset. The data set also comes with the calibration files and ground truth pose. 


## Citation

@INPROCEEDINGS{Geiger2012CVPR,
author = {Andreas Geiger and Philip Lenz and Raquel Urtasun},
title = {Are we ready for Autonomous Driving? The KITTI Vision Benchmark Suite},
booktitle = {Conference on Computer Vision and Pattern	Recognition (CVPR)},
year = {2012}

---


In [54]:
#https://github.com/uoip/monoVO-python

import cv2

# Parameters for lucas kanade optical flow
winSize  = (15,15)
maxLevel = 2
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
0, 0.001

def featureTracking(prev_img, next_img, prev_pts, winSize, maxLevel, criteria):
    term_crit = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 0.1)
    
    # calculate optical flow
    next_pts, status, error = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_pts, winSize, maxLevel, criteria, 0, 0.001)
    
    #TODO: get rid of poins which are outside the frame
    
    return(next_pts, status, error)


#uses FAST as of now, modify parameters as necessary
def featureDetection_fast(img,fast_threshold = 20,nonmaxSuppression = True  ):
    # Initiate FAST object 
    fast = cv2.FastFeatureDetector_create(fast_threshold, nonmaxSuppression)
    # find the keypoints
    points1 = fast.detect(img,)
    return pts

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 1000,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

def featureDetection_ShiTomasi(img, feature_params):
    pts = cv2.goodFeaturesToTrack(img, mask = None, **feature_params)
    return pts





---

## Loading the dataset
Tools courtesy Lee Clement and his group at University of Toronto. 
[pykitti](https://github.com/utiasSTARS/pykitti)



In [64]:
import numpy as np
from collections import namedtuple
import datetime as dt
import glob
import os

frame_range = range(0,200)
base_dir = './KITTI_excerpt1'
sequencepath = basedir+'/sequences/00/'
posepath = basedir+'/poses/'


class load_dataset(object):
    def __init__(self, base_dir, frame_range = None):
        self.base_dir = base_dir
        self.sequence_path = self.base_dir +'/sequences/00/'
        self.pose_path = self.base_dir +'/poses/'
        self.calib_path = self.sequence_path+'calib.txt'
        self.frame_range = frame_range

    def read_calib_file(self, filepath):
        """Read in a calibration file and parse into a dictionary."""
        self.data = {}

        with open(filepath, 'r') as f:
            for line in f.readlines():
                key, value = line.split(':', 1)
                # The only non-float values in these files are dates, which
                # we don't care about anyway
                try:
                    self.data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass

    def load_calib(self):
        """Load and compute intrinsic and extrinsic calibration parameters."""
        # We'll build the calibration parameters as a dictionary, then
        # convert it to a namedtuple to prevent it from being modified later
        data = {}
        
        # Load the calibration file
        self.calib_filedata = read_calib_file(self.calib_path)

        # Create 3x4 projection matrices
        P_rect_00 = np.reshape(filedata['P0'], (3, 4))
        P_rect_10 = np.reshape(filedata['P1'], (3, 4))
        P_rect_20 = np.reshape(filedata['P2'], (3, 4))
        P_rect_30 = np.reshape(filedata['P3'], (3, 4))

        # Compute the camera intrinsics
        data['K_cam0'] = P_rect_00[0:3, 0:3]
        data['K_cam1'] = P_rect_10[0:3, 0:3]
        data['K_cam2'] = P_rect_20[0:3, 0:3]
        data['K_cam3'] = P_rect_30[0:3, 0:3]

        self.calib = namedtuple('CalibData', data.keys())(*data.values())
        self.focal = calib[0][0][0] # Camera focal length
        self.pp = [calib[0][0][2],calib[0][1][2]] #camera pricipal point
        
    def load_timestamps(self):
        """Load timestamps from file."""
        timestamp_file = self.sequence_path+'times.txt'

        # Read and parse the timestamps
        timestamps = []
        with open(timestamp_file, 'r') as f:
            for line in f.readlines():
                t = dt.timedelta(seconds=float(line))
                timestamps.append(t)

        # Subselect the chosen range of frames, if any
        self.timestamps = [timestamps[i] for i in frame_range]

        # print('Found ' + str(len(timestamps)) + ' timestamps...')
        # print('done.')
    
    def load_poses(self):
        """Load ground truth poses from file."""
        # Read and parse the poses
        try:
            self.pose = []
            with open(self.pose_path, 'r') as f:
                for line in f.readlines():
                    T = line.strip().split()
                    xyz = [float(T[3]), float(T[7]), float(T[11])]
                    self.pose.append(xyz)
            #print('done.')
        except FileNotFoundError:
            print('Ground truth poses are not avaialble for sequence')
    
    def load_gray(self):
        im_path = os.path.join(self.sequence_path, 'image_0', '*.png')
        im_files = sorted(glob.glob(imL_path))

        # Subselect the chosen range of frames, if any
        if self.frame_range:
            im_files = [im_files[i] for i in self.frame_range]

        for imfile in im_files:
            im = mpimg.imread(imfile)
            self.gray.append(im)
    

In [ ]:
class movo(object):
    

In [44]:
# Get Absolute Scale between ground truth poses of frame_id and frame_id-1

def getAbsoluteScale(pose, frame_id):  

    prev = pose[frame_id-1]
    curr = pose[frame_id]
    scale = np.sqrt(sum((c-p)**2 for c,p in zip(curr,prev)))
    return(scale)

scale = getAbsoluteScale(pose,2)
print(scale)


0.859327295134


In [ ]:
#  process first two frames

def initProcess(img1, img2):
    
    focal = calib[0][0][0] # Camera focal length
    pp = [calib[0][0][2],calib[0][1][2]] #camera pricipal point
    
    # Process 1st frame
    pts1 = featureDetection_fast(img1)
    
    # Process 2nd frame
    
    pts2, status, error = featureTracking(img1, img2, pts1, winSize, maxLevel, criteria)
    E, mask = cv2.findEssentialMat(pts2, pts1, focal, pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, Rvec, Tvec, mask = cv2.recoverPose(E, pts2, pts1, focal, pp)
    return(img2, pts1, Rvec, Tvec)

In [ ]:
prev_img, prev_pts, curr_Rvec, curr_Tvec = initProcess(img1, img2)

def Process(prev_img, curr_img, prev_pts, frame_id):
    
    curr_pts, status, error = featureTracking(prev_img, curr_img, prev_pts, winSize, maxLevel, criteria)
    
    E, mask = cv2.findEssentialMat(curr_pts, prev_pts, focal, pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, Rvec, Tvec, mask = cv2.recoverPose(E, curr_pts, prev_pts, focal, pp)
    
    absoluteScale = getAbsoluteScale(pose,frame_id)
    if absoluteScale>0.1:
        curr_Tvec += absoluteScale*curr_Rvec.dot(Tvec)
        curr_Rvec = 
        